In [2]:
import sys
sys.path.append("..")

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [16]:
# Import Libraries
from utils import rotate_preserve_size
from loss import angular_loss_mae
import glob
import os
import numpy as np
import cv2
import random

from tensorflow.keras.models import Model
from tensorflow.keras import layers as L
import tensorflow as tf
import os
import pandas as pd
from tensorflow.keras.applications import Xception, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from loguru import logger
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import Adadelta

In [12]:
#Define conv base
# conv_base = Xception(weights="imagenet", include_top=False, input_shape=(299, 299, 3))
conv_base = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(299, 299, 3))
for layer in conv_base.layers:
    layer.trainable = True

16711680/16705208 [==============================] - 0s 0us/step


In [13]:
# Define model
img_input = L.Input(shape=(299, 299, 3))
x = conv_base(img_input)
x = L.Flatten()(x)
x = L.Dense(512, activation="relu")(x)
x = L.BatchNormalization()(x)
x = L.Dense(256, activation="relu")(x)
x = L.BatchNormalization()(x)
x = L.Dense(64, activation="relu")(x)
x = L.BatchNormalization()(x)
y = L.Dense(1, activation="linear")(x)
model = Model(img_input, y)

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 299, 299, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 10, 10, 1280)      4049571   
_________________________________________________________________
flatten_1 (Flatten)          (None, 128000)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               65536512  
_________________________________________________________________
batch_normalization_3 (Batch (None, 512)               2048      
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_4 (Batch (None, 256)               1024

In [36]:
# Batch Generator
class RotGenerator(Sequence):
    def __init__(self, image_dir, batch_size, dim):
        self.files = glob.glob(os.path.join(image_dir, "*.jpg"))
        self.batch_size = batch_size
        self.dim = dim
        
    def __len__(self):
        if len(self.files) % self.batch_size == 0:
            return len(self.files) // self.batch_size
        return len(self.files) // self.batch_size + 1
    
    def __getitem__(self, idx):
        batch_slice = slice(idx * self.batch_size, (idx + 1) * self.batch_size)
        batch_files = self.files[batch_slice]
        
        X = np.zeros(shape=(len(batch_files), self.dim, self.dim, 3))
        y = np.zeros(shape=(len(batch_files), ))
        
        for i, f in enumerate(batch_files):
            # img = cv2.imread(f)
            angle = float(np.random.choice(range(0, 360)))
            img = rotate_preserve_size(f, angle, (self.dim, self.dim))
            
            X[i] = img
            y[i] = angle
        
        return X/255., y
    
    def on_epoch_end(self):
        random.shuffle(self.files)
            

In [39]:
# Batch Generator
class ValidationTestGenerator(Sequence):
    def __init__(self, image_dir, df_label_path, batch_size, dim, mode):
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.dim = dim
        self.mode = mode
        
        df_label = pd.read_csv(df_label_path)
        self.df = df_label[df_label["mode"] == self.mode].reset_index(drop=True)
        
    def __len__(self):
        total = self.df.shape[0]
        if total % self.batch_size == 0:
            return total // self.batch_size
        return total // self.batch_size + 1
    
    def __getitem__(self, idx):
        batch_slice = slice(idx * self.batch_size, (idx + 1) * self.batch_size)
        df_batch = self.df[batch_slice].reset_index(drop=True).copy()
        
        X = np.zeros(shape=(len(df_batch), self.dim, self.dim, 3))
        y = np.zeros(shape=(len(df_batch), ))
        
        for i in range(len(df_batch)):
            angle = df_batch.angle[i]
            path = os.path.join(self.image_dir, df_batch.image[i])
            img = rotate_preserve_size(path, angle, (self.dim, self.dim))
            
            X[i] = img
            y[i] = angle
        
        return X/255., y
    
    def on_epoch_end(self):
        self.df = self.df.sample(frac=1).reset_index(drop=True)
            

In [42]:
# train
model.compile(loss=angular_loss_mae, optimizer=Adadelta(learning_rate=0.1))

train_gen = RotGenerator("/data/chandanp/train2017/", 32, 299)
val_gen = ValidationTestGenerator(image_dir="/data/subhadip/data/validation-test/", 
                                  df_label_path="/data/subhadip/data/validation-test.csv",
                                  batch_size=128, dim=299, mode="valid")
cp = ModelCheckpoint("/data/subhadip/weights/model-en-ang-loss.h5", save_weights_only=False, 
                     save_best_only=True, monitor="loss")
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-5)
es = EarlyStopping(monitor="val_loss", patience=5)
model.fit(train_gen, validation_data=val_gen, epochs=10000, callbacks=[cp, es, reduce_lr])

3/3 [==============================] - ETA: 0s - loss: 79.5585

2022-03-28 01:21:14.243655: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 137319936 exceeds 10% of free system memory.
2022-03-28 01:21:16.820848: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 137319936 exceeds 10% of free system memory.
2022-03-28 01:21:17.309088: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.73GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


3/3 [==============================] - 37s 15s/step - loss: 79.8537 - val_loss: 91.1038
